<a href="https://colab.research.google.com/github/mahdimalverdi/BigData-Exam/blob/main/Exam1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()

In [4]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains
spark = SparkSession.builder.master("local[1]").appName("hw6").getOrCreate()

In [7]:
!wget https://raw.githubusercontent.com/mahdimalverdi/BigData-Exam/main/select_20_features.csv

--2021-06-22 12:48:06--  https://raw.githubusercontent.com/mahdimalverdi/BigData-Exam/main/select_20_features.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44041 (43K) [text/plain]
Saving to: ‘select_20_features.csv’

select_20_features. 100%[===================>]  43.01K  --.-KB/s    in 0.006s  

2021-06-22 12:48:06 (7.31 MB/s) - ‘select_20_features.csv’ saved [44041/44041]



In [11]:
!wget https://raw.githubusercontent.com/mahdimalverdi/BigData-Exam/main/labels.csv

--2021-06-22 12:48:50--  https://raw.githubusercontent.com/mahdimalverdi/BigData-Exam/main/labels.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1112 (1.1K) [text/plain]
Saving to: ‘labels.csv’

labels.csv          100%[===================>]   1.09K  --.-KB/s    in 0s      

2021-06-22 12:48:50 (27.1 MB/s) - ‘labels.csv’ saved [1112/1112]



In [8]:
train_dataset = spark.read.csv("select_20_features.csv",inferSchema=True, header =False)
train_dataset.printSchema()

root
 |-- _c0: double (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: double (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: double (nullable = true)
 |-- _c8: double (nullable = true)
 |-- _c9: double (nullable = true)
 |-- _c10: double (nullable = true)
 |-- _c11: double (nullable = true)
 |-- _c12: double (nullable = true)
 |-- _c13: double (nullable = true)
 |-- _c14: double (nullable = true)
 |-- _c15: double (nullable = true)
 |-- _c16: double (nullable = true)
 |-- _c17: double (nullable = true)
 |-- _c18: double (nullable = true)
 |-- _c19: double (nullable = true)



In [12]:
lable_dataset = spark.read.csv("labels.csv",inferSchema=True, header =True)
lable_dataset.printSchema()

root
 |-- lable: integer (nullable = true)



In [13]:
from pyspark.sql.types import StructType, StructField, LongType

def with_column_index(sdf): 
    new_schema = StructType(sdf.schema.fields + [StructField("ColumnIndex", LongType(), False),])
    return sdf.rdd.zipWithIndex().map(lambda row: row[0] + (row[1],)).toDF(schema=new_schema)

df1_ci = with_column_index(lable_dataset)
df2_ci = with_column_index(train_dataset)
join_on_index = df1_ci.join(df2_ci, df1_ci.ColumnIndex == df2_ci.ColumnIndex, 'inner').drop("ColumnIndex")

In [14]:
def getHeaders(n):
  return n.name

headers = list(map(getHeaders,train_dataset.schema.fields))

In [20]:
from pyspark.ml.feature import VectorAssembler
import pyspark.sql.functions as F

assembler = VectorAssembler(inputCols=headers,outputCol = 'Attributes')

output = assembler.transform(join_on_index)
finalized_data = output.select("Attributes","lable")
finalized_data.show()

+--------------------+-----+
|          Attributes|lable|
+--------------------+-----+
|[0.75,1.0,0.5,0.5...|    1|
|[0.75,1.0,0.5,0.5...|    1|
|[1.0,1.0,0.25,0.5...|    3|
|[0.75,1.0,0.5,0.5...|    1|
|[0.75,1.0,0.5,0.5...|    1|
|[1.0,0.5,0.5,0.75...|    2|
|[0.75,0.5,0.5,0.2...|    0|
|[1.0,0.25,0.25,1....|    3|
|[0.75,1.0,0.5,0.5...|    1|
|[0.25,0.5,0.5,0.5...|    1|
|[0.75,1.0,0.5,0.5...|    1|
|[0.75,1.0,0.5,0.5...|    1|
|[1.0,0.75,0.5,0.5...|    3|
|[0.25,0.75,0.5,0....|    2|
|[0.75,1.0,0.5,0.5...|    1|
|[1.0,0.5,0.25,0.5...|    1|
|[0.75,1.0,0.5,0.5...|    1|
|[1.0,0.5,0.5,1.0,...|    0|
|[0.75,0.5,0.5,0.2...|    1|
|[0.25,0.5,0.5,0.5...|    1|
+--------------------+-----+
only showing top 20 rows



In [32]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import RegressionEvaluator

def evaluate(result):
  evaluator = MulticlassClassificationEvaluator(
      labelCol="lable", predictionCol="prediction", metricName="accuracy")
  accuracy = evaluator.evaluate(result)
  print("Test Error = %g " % (1.0 - accuracy))

In [33]:
from pyspark.ml.regression import DecisionTreeRegressor
regressor = DecisionTreeRegressor(featuresCol = 'Attributes', labelCol = 'lable')
data  = finalized_data.randomSplit([10.0, 1.0], 24)
regressor = regressor.fit(data[0])
result = regressor.transform(data[1])
result.show()
evaluate(result)

+--------------------+-----+------------------+
|          Attributes|lable|        prediction|
+--------------------+-----+------------------+
|[0.75,1.0,0.5,0.5...|    1|0.9775280898876404|
|[1.0,0.5,0.25,0.5...|    1|1.0222222222222221|
|[1.0,0.5,0.5,0.75...|    2|1.8846153846153846|
|[1.0,0.5,0.25,0.5...|    1|1.0222222222222221|
|[1.0,0.5,0.5,0.75...|    2|1.8846153846153846|
|[0.25,0.75,0.5,0....|    2|1.8846153846153846|
|[0.5,1.0,1.0,1.0,...|    2|               0.5|
|[0.75,1.0,0.5,0.5...|    1|0.9775280898876404|
|[1.0,1.0,0.75,1.0...|    3|0.6666666666666666|
|[0.5,0.5,0.75,0.7...|    1|1.0222222222222221|
|[1.0,0.5,0.5,0.75...|    2|1.8846153846153846|
|[1.0,0.25,1.0,0.7...|    3|2.5714285714285716|
|[0.75,0.5,0.75,1....|    2|               1.8|
|[0.75,1.0,0.5,0.5...|    1|0.9775280898876404|
|[1.0,0.5,0.5,0.75...|    2|1.8846153846153846|
|[1.0,0.5,0.25,0.5...|    1|1.0222222222222221|
|[0.5,0.25,0.5,0.7...|    1|1.8846153846153846|
|[0.5,0.75,0.5,0.2...|    1|1.8846153846